In [3]:
import pandas as pd
import pyodbc as py
import sys


path = 'C:\Anup\Test\TestMember.txt'
server = 'XX.XX.XX.XX'
username = 'DADevUser'
password = 'XXXXXX'
database = 'DataAnalytics_Datamart'
filetypeid = '32'
csize = 10000

#path = sys.argv[1]
#server = sys.argv[2]
#username = sys.argv[3]
#password = sys.argv[4]
#database = sys.argv[5]
#filetypeid = sys.argv[6]
#csize = sys.argv[7]


connStr = (r'DRIVER={ODBC Driver 17 for SQL Server};' +
           r"Integrated Security=True;" +
           r'SERVER=' + server +
           r';UID=' + username +
           r';PWD=' + password +
           r';DSN=MSSQL-PYTHON' +
           r';DATABASE=' + database + ';'
           )
# =============================================================================
# CONNECT TO THE DB
# =============================================================================
cnxn = py.connect(connStr, autocommit=True)


SQLQUERY1 = ("SELECT InputtableName,FileTypeID, Width = STUFF(( ")
SQLQUERY1 = SQLQUERY1 + ("SELECT  ',' + convert(varchar(3),[Length]) ")
SQLQUERY1 = SQLQUERY1 + ("FROM [InboundExtractLayout_FixedWidth] As T2 ")
SQLQUERY1 = SQLQUERY1 + ("WHERE T2.FileTypeID = T1.FileTypeID ")
SQLQUERY1 = SQLQUERY1 + ("order by [order] ")
SQLQUERY1 = SQLQUERY1 + ("FOR XML PATH (''), TYPE).value('.', 'varchar(max)'), 1, 1, '') ")
SQLQUERY1 = SQLQUERY1 + (",Count(1) Column_Cnt, ")
SQLQUERY1 = SQLQUERY1 + ("Parameter = STUFF(( ")
SQLQUERY1 = SQLQUERY1 + ("SELECT ',' + '?' ")
SQLQUERY1 = SQLQUERY1 + ("FROM [InboundExtractLayout_FixedWidth] As  T2 ")
SQLQUERY1 = SQLQUERY1 + ("WHERE T2.FileTypeID = T1.FileTypeID ")
SQLQUERY1 = SQLQUERY1 + ("order by [order] ")
SQLQUERY1 = SQLQUERY1 + ("FOR XML PATH (''), TYPE).value('.', 'varchar(max)'), 1, 1, '') ")
SQLQUERY1 = SQLQUERY1 + ("FROM [InboundExtractLayout_FixedWidth] As T1 ")
SQLQUERY1 = SQLQUERY1 + ("where T1.FileTypeID = ") + filetypeid
SQLQUERY1 = SQLQUERY1 + ("GROUP BY FileTypeID, InputtableName ")
print(SQLQUERY1)
df = pd.read_sql(SQLQUERY1, cnxn)
#print(df)


strwidths = df.loc[0,'Width']
widths = strwidth.split(',')
widths = list(map(int, widths))

#print(widths)

# Using Pandas with a column specification


for data in pd.read_fwf(path, chunksize = csize, header=None, widths=widths):
    data = data.fillna(value=" ")
    #print(data)
    cursor = cnxn.cursor()
    cursor.fast_executemany = False
    Insert_stmt = "INSERT INTO "  +  database + ".." + df.loc[0,'InputtableName'] + " values(" + df.loc[0,'Parameter'] + ")"
    #print(Insert_stmt)
    cursor.executemany(Insert_stmt, data.values.tolist())
    cursor.commit()
    cursor.close()


cnxn.close()


SELECT InputtableName,FileTypeID, Width = STUFF(( SELECT  ',' + convert(varchar(3),[Length]) FROM [InboundExtractLayout_FixedWidth] As T2 WHERE T2.FileTypeID = T1.FileTypeID order by [order] FOR XML PATH (''), TYPE).value('.', 'varchar(max)'), 1, 1, '') ,Count(1) Column_Cnt, Parameter = STUFF(( SELECT ',' + '?' FROM [InboundExtractLayout_FixedWidth] As T2 WHERE T2.FileTypeID = T1.FileTypeID order by [order] FOR XML PATH (''), TYPE).value('.', 'varchar(max)'), 1, 1, '') FROM [InboundExtractLayout_FixedWidth] As T1 where T1.FileTypeID = 32GROUP BY FileTypeID,InputtableName 


'\nstrwidths = df.loc[0,\'Width\']\nwidths = strwidth.split(\',\')\nwidths = list(map(int, widths))\n\n#print(widths)\n\n# Using Pandas with a column specification\n\n\nfor data in pd.read_fwf(path, chunksize = csize, header=None, widths=widths):\n    data = data.fillna(value=" ")\n    #print(data)\n    cursor = cnxn.cursor()\n    cursor.fast_executemany = False\n    Insert_stmt = "INSERT INTO "  +  database + ".." + df.loc[0,\'InputtableName\'] + " values(" + df.loc[0,\'Parameter\'] + ")"\n    #print(Insert_stmt)\n    cursor.executemany(Insert_stmt, data.values.tolist())\n    cursor.commit()\n    cursor.close()\n\n\ncnxn.close()\n'